In [1]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 22.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
# load the library
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
import torch
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# initialize the parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "google/flan-t5-base"
tokenizer_name_or_path = "google/flan-t5-base"

checkpoint_name = ""
text_column = "sentence"
label_column = "text_label"
max_length = 256
lr = 3e-4
num_epochs = 3
batch_size = 16
model_path = './drive/MyDrive/colab/Flan-T5-Lora/model/flan-t5-COVID-QA'

In [7]:
# create the model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = T5ForConditionalGeneration.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3560841867092814


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [8]:
# load the dataset
dataset = load_dataset("covid_qa_deepset")

columns_to_remove = ["document_id", "context", "is_impossible", "id"]
dataset = dataset.remove_columns(columns_to_remove)

dataset = dataset['train'].train_test_split(test_size=0.2)
dataset['validation'] = dataset['test']
del dataset['test']

dataset = dataset.map(lambda x: {'answer': x['answers']['text'][0]})
dataset = dataset.remove_columns("answers")
print(dataset)

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Map:   0%|          | 0/1615 [00:00<?, ? examples/s]

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1615
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 404
    })
})


In [9]:
dataset['train'][0]

{'question': 'Approximately how many people died during the 1918-1919 influenza pandemic?',
 'answer': 'During the 1918-1919 influenza pandemic, between 21 and 25 million people died of influenza worldwide.'}

In [10]:
# data preprocessing
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

prefix = "answer the question: "

def preprocess_function(examples):
    """add prefix to the sentences, tokenize the text, and set the labels"""
    # the "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding='max_length')

    # the "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], max_length=max_length, truncation=True, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# map the preprocessing function across our dataset
tokenized_dataset = dataset.map(
    preprocess_function,
    remove_columns=dataset["train"].column_names,
    batched=True
)

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1615 [00:00<?, ? examples/s]

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

In [11]:
print(train_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1615
})


In [12]:
print(eval_dataset)
print(train_dataloader)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 404
})


In [13]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [16]:
# training and evaluation
model.to(device)
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for step, batch in enumerate(tqdm(train_dataloader)):
        # batch = data_collator.collate_batch(batch)
        # move data to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # clear gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # backward pass and optimization
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

    # print each epoch's average loss
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    # evaluation
    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            eval_loss += loss.item()

    avg_eval_loss = eval_loss / len(eval_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Eval Loss: {avg_eval_loss:.4f}")

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

100%|██████████| 404/404 [00:51<00:00,  7.92it/s]


Epoch 1/3, Average Loss: 1.5097
Epoch 1/3, Average Eval Loss: 0.4969


100%|██████████| 404/404 [00:50<00:00,  7.94it/s]


Epoch 2/3, Average Loss: 0.5530
Epoch 2/3, Average Eval Loss: 0.4130


100%|██████████| 404/404 [00:50<00:00,  7.95it/s]


Epoch 3/3, Average Loss: 0.4845
Epoch 3/3, Average Eval Loss: 0.4060


('./drive/MyDrive/colab/main/model/flan-t5-COVID-QA/tokenizer_config.json',
 './drive/MyDrive/colab/main/model/flan-t5-COVID-QA/special_tokens_map.json',
 './drive/MyDrive/colab/main/model/flan-t5-COVID-QA/spiece.model',
 './drive/MyDrive/colab/main/model/flan-t5-COVID-QA/added_tokens.json')

In [18]:
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./drive/MyDrive/colab/Flan-T5-Lora/model/flan-t5-COVID-QA/tokenizer_config.json',
 './drive/MyDrive/colab/Flan-T5-Lora/model/flan-t5-COVID-QA/special_tokens_map.json',
 './drive/MyDrive/colab/Flan-T5-Lora/model/flan-t5-COVID-QA/spiece.model',
 './drive/MyDrive/colab/Flan-T5-Lora/model/flan-t5-COVID-QA/added_tokens.json')

### ask questions

In [2]:
# original model
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

input_text = "What is DC-GENR and where is it expressed?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
print(input_ids)

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tensor([[  363,    19,  5795,    18, 18464,   448,    11,   213,    19,    34,
          7103,    58,     1]], device='cuda:0')


c:\Users\31598\.conda\envs\pytorch_gpu\Lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> genital marrow</s>


In [3]:
from peft import PeftConfig, PeftModel
config = PeftConfig.from_pretrained("./model/flan-t5-covid-lora")
model = T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, "./model/flan-t5-covid-lora")

tokenizer = T5Tokenizer.from_pretrained(config.base_model_name_or_path)
input_text = "What measures have been implemented to slow the spread of COVID-19?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(input_ids)

outputs = model.generate(input_ids=input_ids)
print(tokenizer.decode(outputs[0]))

c:\Users\31598\.conda\envs\pytorch_gpu\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\31598\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned

tensor([[ 363, 3629,   43,  118, 6960,   12, 2684,    8, 3060,   13, 2847, 7765,
          308, 4481,   58,    1]])
<pad> e-mails to a large number of people</s>


In [4]:
input_text = 'What efforts are being made to develop vaccines and treatments for COVID-19?'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids=input_ids, max_new_tokens=30)
print(tokenizer.decode(outputs[0]))

<pad> Vaccines and treatments for COVID-19</s>


In [5]:
input_text = 'What is DC-GENR and where is it expressed?'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids=input_ids, max_new_tokens=30)
print(tokenizer.decode(outputs[0]))

<pad> DC-GENR</s>
